<a href="https://colab.research.google.com/github/Heineken97/Transferencia-de-Estilo-Basic/blob/main/StyleTransfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import functools
import os

from matplotlib import gridspec
import matplotlib.pylab as plt
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

print("TensorFlow Version: ", tf.__version__)
print("Tensorflow Hub Version: ",hub.__version__)
print("Modo Eager Activado: ", tf.executing_eagerly() )
print("GPU Disponible: ", tf.config.list_physical_devices('GPU'))

TensorFlow Version:  2.8.2
Tensorflow Hub Version:  0.12.0
Modo Eager Activado:  True
GPU Disponible:  []


In [3]:
"Retorna imagen cortada en cuadrado"
def crop_center(image):
  shape = image.shape
  new_shape = min(shape[1], shape[2])
  offset_y = max(shape[1] - shape[2], 0) // 2
  offset_x = max(shape[2] - shape[1], 0) // 2
  image = tf.image.crop_to_bounding_box(image, offset_y, offset_x, new_shape, new_shape)
  return image

@functools.lru_cache(maxsize=None)
def load_image(image_url, image_size=(256, 256), preserve_aspect_ratio=True):
  "Carga y preprocesa las imagenes"
  image_path = tf.keras.utils.get_file(os.path.basename(image_url)[-128:], image_url)
  "Carga y convierte a float32NumpyArray, añade batch de dimension y normaliza rango [0,1] "
  img = tf.io.decode_image(tf.io.read_file(image_path),channels=3, dtype=tf.float32)[tf.newaxis, ...]
  img = crop_center(img)
  img = tf.image.resize(img, image_size, preserve_aspect_ratio=True)
  return img

def show_n(images, titles=('',)):
  n = len(images)
  image_sizes = [image.shape[1] for image in images]
  w = (image_sizes[0] * 6) // 320
  plt.figure(figsize=(w * n, w))
  gs = gridspec.GridSpec(1, n, width_ratios=image_sizes)
  for i in range(n):
    plt.subplot(gs[i])
    plt.imshow(images[i][0], aspect='equal')
    plt.axis('off')
    plt.title(titles[i] if len(titles) > i else '')
  plt.show()


In [ ]:
content_image_url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/f/fd/Golden_Gate_Bridge_from_Battery_Spencer.jpg/640px-Golden_Gate_Bridge_from_Battery_Spencer.jpg'  
style_image_url = 'https://upload.wikimedia.org/wikipedia/commons/0/0a/The_Great_Wave_off_Kanagawa.jpg'  
output_image_size = 384  

content_img_size = (output_image_size, output_image_size)
style_img_size = (256, 256) 

content_image = load_image(content_image_url, content_img_size)
style_image = load_image(style_image_url, style_img_size)
style_image = tf.nn.avg_pool(style_image, ksize=[3,3], strides=[1,1], padding='SAME')
show_n([content_image, style_image], ['Content image', 'Style image'])


In [6]:
hub_handle = 'https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2'
hub_module = hub.load(hub_handle)
outputs = hub_module(tf.constant(content_image), tf.constant(style_image))
stylized_image = outputs[0] 

In [38]:
content_urls = dict(
  odin='https://scontent.fsyq7-1.fna.fbcdn.net/v/t1.6435-9/120246891_3994040037289334_7412756626256152626_n.jpg?_nc_cat=101&ccb=1-7&_nc_sid=09cbfe&_nc_ohc=FC2Ie3hixl8AX8XAx_I&tn=uFkqCULjWlhWhNr3&_nc_ht=scontent.fsyq7-1.fna&oh=00_AT9qw9lwg_EZ9zS6REAnRXKN48pX_ld-RJpegfJGI_3shQ&oe=62F8CF16',
  abuelos='https://scontent.fsyq7-1.fna.fbcdn.net/v/t1.6435-9/123073343_3051369821634094_8325041358886777189_n.jpg?_nc_cat=105&ccb=1-7&_nc_sid=730e14&_nc_ohc=SE3SATL6A4kAX_QzQRR&_nc_ht=scontent.fsyq7-1.fna&oh=00_AT_lKg9GFAMWo_SHWvlb7SMeOTOP8OvQ49G3-EKWwVj9YQ&oe=62F9BABE',
  joseph='https://scontent.fsyq7-1.fna.fbcdn.net/v/t31.18172-8/12362899_916434981739605_9026491197154873040_o.jpg?_nc_cat=109&ccb=1-7&_nc_sid=730e14&_nc_ohc=cf8xwGTjXAoAX8HDDaa&_nc_ht=scontent.fsyq7-1.fna&oh=00_AT942wmb0EJeYMR5N9Lp60ljf7zxMTQB6ffbBO1bnfkyEw&oe=62F6A9CC',
  )
style_urls = dict(
  lapiz_de_color='https://i.ibb.co/XpJnzLG/jason-leung-w-Hdd-Vi-Tm-Sv-A-unsplash.jpg',
  kanagawa_great_wave='https://upload.wikimedia.org/wikipedia/commons/0/0a/The_Great_Wave_off_Kanagawa.jpg',
  kandinsky_composition_7='https://upload.wikimedia.org/wikipedia/commons/b/b4/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg',
  hubble_pillars_of_creation='https://upload.wikimedia.org/wikipedia/commons/6/68/Pillars_of_creation_2014_HST_WFC3-UVIS_full-res_denoised.jpg',
  van_gogh_starry_night='https://upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg/1024px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg',
  turner_nantes='https://upload.wikimedia.org/wikipedia/commons/b/b7/JMW_Turner_-_Nantes_from_the_Ile_Feydeau.jpg',
  munch_scream='https://upload.wikimedia.org/wikipedia/commons/c/c5/Edvard_Munch%2C_1893%2C_The_Scream%2C_oil%2C_tempera_and_pastel_on_cardboard%2C_91_x_73_cm%2C_National_Gallery_of_Norway.jpg',
  picasso_demoiselles_avignon='https://upload.wikimedia.org/wikipedia/en/4/4c/Les_Demoiselles_d%27Avignon.jpg',
  picasso_violin='https://upload.wikimedia.org/wikipedia/en/3/3c/Pablo_Picasso%2C_1911-12%2C_Violon_%28Violin%29%2C_oil_on_canvas%2C_Kr%C3%B6ller-M%C3%BCller_Museum%2C_Otterlo%2C_Netherlands.jpg',
  picasso_bottle_of_rum='https://upload.wikimedia.org/wikipedia/en/7/7f/Pablo_Picasso%2C_1911%2C_Still_Life_with_a_Bottle_of_Rum%2C_oil_on_canvas%2C_61.3_x_50.5_cm%2C_Metropolitan_Museum_of_Art%2C_New_York.jpg',
  fire='https://upload.wikimedia.org/wikipedia/commons/3/36/Large_bonfire.jpg',
  derkovits_woman_head='https://upload.wikimedia.org/wikipedia/commons/0/0d/Derkovits_Gyula_Woman_head_1922.jpg',
  amadeo_style_life='https://upload.wikimedia.org/wikipedia/commons/8/8e/Untitled_%28Still_life%29_%281913%29_-_Amadeo_Souza-Cardoso_%281887-1918%29_%2817385824283%29.jpg',
  derkovtis_talig='https://upload.wikimedia.org/wikipedia/commons/3/37/Derkovits_Gyula_Talig%C3%A1s_1920.jpg',
  amadeo_cardoso='https://upload.wikimedia.org/wikipedia/commons/7/7d/Amadeo_de_Souza-Cardoso%2C_1915_-_Landscape_with_black_figure.jpg'
)

content_image_size = 384
style_image_size = 256
content_images = {k: load_image(v, (content_image_size, content_image_size)) for k, v in content_urls.items()}
style_images = {k: load_image(v, (style_image_size, style_image_size)) for k, v in style_urls.items()}
style_images = {k: tf.nn.avg_pool(style_image, ksize=[3,3], strides=[1,1], padding='SAME') for k, style_image in style_images.items()}

155648/140933 [=================================] - 0s 2us/step


In [ ]:
content_name = 'abuelos'
style_name = 'amadeo_style_life' 
stylized_image = hub_module(tf.constant(content_images[content_name]),tf.constant(style_images[style_name]))[0]
show_n([content_images[content_name], style_images[style_name], stylized_image],titles=['Imagen Original', 'Estilo de imagen', 'Imagen con estilo'])